In [1]:
import subprocess
import glob
import re
import pandas as pd
from pathlib import Path
import bisect
from transformers import AutoTokenizer, AutoModel
import torch
import tensorflow as tf
import numpy as np
import csv

In [2]:
def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]


In [17]:

def extractInfos(f) :
  comment,repository,commit,commitType="","","",""

  with open(file, "r",encoding="utf-8", errors="ignore") as f:
        
        for i, line in enumerate(f):

          #extract the commit information from the first line
          if i==0:
            infos=line.split(',')
            repository=infos[0]
            commit=infos[4]
            commitType=infos[3]
            comment= infos[-1:][0]
          else:
            if ("diff --git" in line) or ("Signed-off-by" in line) or ("Reported-by" in line) or ("Acked-by" in line):
              break
            else:
              if line !="\n":
                comment += line
  return (repository,commit,commitType,comment)

In [18]:
def liste_fichiers(file):
  liste=[]
  repository=""
  commit=""
  commitType=""
  newFileName=""
  comment=""

  with open(file, "r", encoding="utf-8", errors="ignore") as f:
        repository,commit,commitType,comment= extractInfos(file)
        for i, line in enumerate(f):
          if "diff --git" in line:
            newFileName=line.rsplit('/', 1)[1]
            newFileLine=i
            if newFileName != "" :
              liste.append((f.name,repository,commit,commitType,newFileLine,newFileName[:-1],comment))

  return liste

In [19]:
def lines_that_contain(file,string):
  with open(file, "r", encoding="utf-8", errors="ignore") as fp:
    return [(i,line) for i,line in enumerate(fp) if string in line]

In [20]:
def split_Chars(word):
    return [char for char in word]

In [21]:
#get the method's code
def process_file(filename, line_num):
  
    code = ""
    cnt_braket = 0
    cnt_parentheses=0
    found_start = False
    found_end = False

    startbraket=0
    startbraket2=0

    with open(filename, "r", encoding="utf-8", errors="ignore") as f:
        for i, line in enumerate(f):
            if(i >= line_num):
              code += line
              if line.count("(") > 0:
                cnt_parentheses += line.count("(")
               
              if line.count(")") > 0:
                cnt_parentheses -= line.count(")")
                
              if cnt_parentheses == 0 :
                  startbraket=i
                  break
                
    with open(filename, "r", encoding="utf-8", errors="ignore") as f:
        for i, line in enumerate(f):
            if(i >= startbraket):
              
              code += line
              test=False
            
              if line == startbraket :
                test= line.count("{")>0  
              else:
                if line !="" and line !="\n":  
                  if split_Chars(line)[0]=="+" or  split_Chars(line)[0]=="-" :
                    if len(split_Chars(line)) >1 : test= split_Chars(line)[1]=="{"
                  else :
                    test= split_Chars(line)[0]=="{"
                    
              if test:       
                    found_start = True
                    cnt_braket += line.count("{")
                    startbraket2=i
                    break
              else :
               if line !="" and line !="\n" and i!=startbraket:
                 return ""

    with open(filename, "r", encoding="utf-8", errors="ignore") as f:
        if found_start == True:
          
          for i, line in enumerate(f):
              if(i > startbraket2):
                code += line
                if line.count("{") > 0:
                  cnt_braket += line.count("{")
                if line.count("}") > 0:
                  cnt_braket -= line.count("}")
                if cnt_braket ==0 and found_start:
                  found_end = True
                  return code
    return ""



In [22]:
# Init the model
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

In [10]:
# assign directory
directory = 'D:\PFEMaster\Dataset et pretraitement\Dataset_VCCPlus'
 
# iterate over files in that directory
files = Path(directory).glob('*.txt')

In [11]:
#initialiser les variables
fct_code=[]
fct_names=[]
tableRef=[]
classification=[]
embeddings_same_size=[]

In [23]:
for file in files:
    code=""
    #get the code without the annotations
    with open(file, "r", encoding="utf-8", errors="ignore") as fp:
        for line in fp:
            if (line[0]=="+" or line[0]=="-"):
                code+=line[1:]
            else:
              code+=line
            
    #get the modified methods' names
    fct_names= re.findall(r'(?<=(?<=int\s)|(?<=void\s)|(?<=string\s)|(?<=double\s)|(?<=float\s)|(?<=\w\s)|(?<=char\s)).*?(?=\s?\()', code)
    fct_names=list(dict.fromkeys(fct_names))
    
    #get a list of the modified files
    fichiers_modif=liste_fichiers(file)
    position_fichiers=[f[4] for f in fichiers_modif]
    
    #get the  modified methods' code
    for function in fct_names:
      for line in lines_that_contain(file,function):
        code = process_file(file,line[0])
        if code !="":
            bisect.insort(position_fichiers, line[0])
            index_fichier= position_fichiers[position_fichiers.index(line[0])-1]
            position_fichiers.remove(line[0])

            infos=[f for f in fichiers_modif if f[4]==index_fichier][0]
            fct_code.append((infos[0],infos[1],infos[2],infos[3],infos[4],infos[5],line[0],function,code,infos[6]))

    fct_names.clear()


In [24]:
#tokenize the methods
for f in fct_code :
   # Tokenization
   nl_tokens=tokenizer.tokenize(f[9])
   code_tokens=tokenizer.tokenize(f[8])
   tokens=nl_tokens+[tokenizer.sep_token]+code_tokens

   #verifier que la serie ne depasse pas 510
   if len(tokens) > 510:
      fractions = list(divide_chunks(code_tokens,509-len(nl_tokens)))
      for fract in fractions:
          fract= [tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+fract+[tokenizer.sep_token]
          tokens_ids=tokenizer.convert_tokens_to_ids(fract)
          tableRef.append((f[0],f[1],f[2],f[3],f[4],f[5],tokens_ids))
   else:   
      tokens= [tokenizer.cls_token]+tokens+[tokenizer.sep_token]
      # Convert tokens to ids
      tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
      tableRef.append((f[0],f[1],f[2],f[3],f[4],f[5],tokens_ids))

Token indices sequence length is longer than the specified maximum sequence length for this model (841 > 512). Running this sequence through the model will result in indexing errors


In [25]:
with open('D:/Tokens/CodeBert/CodeBert_tableTokens_methodes.csv', "w") as f:
    wr = csv.writer(f)
    wr.writerows(tableRef)

In [26]:
max_value=max([len(embed[6]) for embed in tableRef ]) 
print(max_value)

512


In [ ]:
iter=0
with open("D:/Embeddings/CodeBert/CodeBert_methodesEmbeddings.csv", "w") as f:
    for embed in tableRef :

      tokens_same_size=np.pad(embed[6], (0, max_value-len(embed[6])), 'constant')
      context_embeddings=model(torch.tensor(tokens_same_size).unsqueeze(0))
      t = torch.flatten(context_embeddings.pooler_output)
      arr=torch.Tensor.numpy(t.detach())
        
      wr = csv.writer(f)
      wr.writerow(arr)
      iter+=1
      print(iter)

1
2
3
4
5
6
7
